In [3]:
import pandas as pd
train=pd.read_csv(r"c:\Users\Arun\Downloads\archive\walmart_train.csv")
test=pd.read_csv(r"C:\Users\Arun\Downloads\archive\walmart_test.csv")

train=train.dropna()

x_train=(train.drop("TripType",axis=1)).drop("Upc",axis=1)
x_test=test.drop("Upc",axis=1)
y_train=train["TripType"]
print(x_train.isnull().value_counts())
x_train.head()
print(x_test.head())

VisitNumber  Weekday  ScanCount  DepartmentDescription  FinelineNumber
False        False    False      False                  False             642925
Name: count, dtype: int64
   VisitNumber Weekday  ScanCount DepartmentDescription  FinelineNumber
0            1  Friday          1                 SHOES          3002.0
1            1  Friday          1                 DAIRY          1526.0
2            1  Friday          1                 DAIRY          1431.0
3            1  Friday          1     GROCERY DRY GOODS          3555.0
4            2  Friday          1           DSD GROCERY          4408.0


In [4]:
# from ydata_profiling import ProfileReport
# report=ProfileReport(train,title="report",explorative =True)
# report.to_notebook_iframe()

In [5]:
from sklearn.preprocessing import StandardScaler,PowerTransformer,OneHotEncoder,LabelEncoder
from sklearn.compose import make_column_transformer
print(x_train.columns)
transform=make_column_transformer(
    (StandardScaler(),[0,4]),
    (PowerTransformer(),[2]),
    (OneHotEncoder(),[1,3])
)
x_train=transform.fit_transform(x_train)
x_test=transform.fit_transform(x_test)
le=LabelEncoder()
y_train=le.fit_transform(y_train)



Index(['VisitNumber', 'Weekday', 'ScanCount', 'DepartmentDescription',
       'FinelineNumber'],
      dtype='object')


In [7]:
from xgboost import XGBClassifier

model=XGBClassifier(tree_method="gpu_hist",
                    predictor="gpu_predictor",
                    gpu_id=0,
                    
)
model.fit(x_train,y_train)
y_pred=model.predict(x_test)

C:\Users\Arun\AppData\Roaming\Python\Python310\site-packages\xgboost\training.py:183: UserWarning: [22:11:48] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\Arun\AppData\Roaming\Python\Python310\site-packages\xgboost\training.py:183: UserWarning: [22:11:48] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\Arun\AppData\Roaming\Python\Python310\site-packages\xgboost\training.py:183: UserWarning: [22:11:48] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\Arun\AppD

In [8]:
from sklearn.model_selection import cross_val_score
scores= cross_val_score(model,x_train,y_train,scoring="accuracy",n_jobs=-1)
print(scores)


[0.06377882 0.05989812 0.05813275 0.04731501 0.09333126]


In [9]:
print(scores.mean())


0.06449119259633705


In [10]:
from lightgbm import LGBMClassifier

model_log = LGBMClassifier(
    device='gpu',         # ✅ GPU enable
    gpu_platform_id=0,    # Optional
    gpu_device_id=0       # Optional
)

model_log.fit(x_train,y_train)
mpred=model_log.predict(x_test)

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 684
[LightGBM] [Info] Number of data points in the train set: 642925, number of used features: 77
[LightGBM] [Info] Using requested OpenCL platform 0 device 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2050, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 4 dense feature groups (2.45 MB) transferred to GPU in 0.009122 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score -4.545875
[LightGBM] [Info] Start training from score -6.575843
[LightGBM] [Info] Start training from score -4.045393
[LightGBM] [Info] Start training from score -5.241665
[LightGBM] [Info] Start training from score -3.323256
[LightGBM] [Info] Start training from score -3.340760
[LightGBM] [Info] Start training from score -3.648346
[LightGBM] [Info] Start training fr

In [ ]:
scores= cross_val_score(model_log,x_train,y_train,scoring="accuracy",cv=3,n_jobs=-1)


<built-in method mean of numpy.ndarray object at 0x0000028CCF8AD770>


In [13]:
print(scores.mean())

0.08827784485563299
